### Сбор данных с сайта hh.ru

In [ ]:
#!pip install beautifulsoup4 lxml

Загружались вакансии за последний месяц по ключенвым словам: data science, deep learning и т.д.
Из скачанных данных были отобраны наиболее интересные поля. Сбор данных был разделен на 2 этапа из-за ограничения API.

In [1]:
!ls

data  gathering.py  hh_data_gathering.ipynb  parsers  scrappers  storages


**1. Поиск и загрузка открытых вакансий**

In [19]:
# --search - строка поиска вакансий
# --pages - число страниц
# число вакансий на странице, max = 100 - ограничение API

In [97]:
!mkdir data

In [98]:
!python3 -m gathering gather --search '!"data science" OR !"data scientist" OR !"machine learning" OR !"Deep Learning" OR !"Data Analysis"' \
--pages 20 --file 'data/hh_scrapped_data_part1.txt'

INFO:__main__:Work started
INFO:__main__:Gather process
INFO:scrappers.hh_scrapper:Getting data from hh.ru...
INFO:scrappers.hh_scrapper:search_text: !"data science" OR !"data scientist" OR !"machine learning" OR !"Deep Learning" OR !"Data Analysis"
INFO:scrappers.hh_scrapper:per_page: 100
INFO:scrappers.hh_scrapper:pages: 20
100%|███████████████████████████████████████| 1222/1222 [02:06<00:00,  9.64it/s]
INFO:scrappers.hh_scrapper:Save 1222 vacancies from hh.ru to the data/hh_scrapped_data_part1.txt file
INFO:__main__:Work ended


In [99]:
!python3 -m gathering gather --search '!"машинное обучение" OR !"анализ данных" OR !"нейронные сети"' \
--pages 20 --file 'data/hh_scrapped_data_part2.txt'

INFO:__main__:Work started
INFO:__main__:Gather process
INFO:scrappers.hh_scrapper:Getting data from hh.ru...
INFO:scrappers.hh_scrapper:search_text: !"машинное обучение" OR !"анализ данных" OR !"нейронные сети"
INFO:scrappers.hh_scrapper:per_page: 100
INFO:scrappers.hh_scrapper:pages: 20
100%|███████████████████████████████████████| 2000/2000 [03:13<00:00, 10.36it/s]
INFO:scrappers.hh_scrapper:Save 2000 vacancies from hh.ru to the data/hh_scrapped_data_part2.txt file
INFO:__main__:Work ended


**2. Преобразование загруженных данных в табличный формат**

In [100]:
!python3 -m gathering transform --filefrom 'data/hh_scrapped_data_part1.txt' --fileto 'data/hh_data_part1.csv'

INFO:__main__:Work started
INFO:__main__:transform
INFO:parsers.hh_parser:Transformed data saved to the data/hh_data_part1.csv file
INFO:__main__:Work ended


In [101]:
!python3 -m gathering transform --filefrom 'data/hh_scrapped_data_part2.txt' --fileto 'data/hh_data_part2.csv'

INFO:__main__:Work started
INFO:__main__:transform
INFO:parsers.hh_parser:Transformed data saved to the data/hh_data_part2.csv file
INFO:__main__:Work ended


In [102]:
!ls -lh data

total 88M
-rw-rw-r-- 1 mariya mariya 4,4M сен 30 20:52 hh_data_part1.csv
-rw-rw-r-- 1 mariya mariya 6,8M сен 30 20:52 hh_data_part2.csv
-rw-rw-r-- 1 mariya mariya  32M сен 30 20:45 hh_scrapped_data_part1.txt
-rw-rw-r-- 1 mariya mariya  45M сен 30 20:50 hh_scrapped_data_part2.txt


**3. Объединение загруженных данных**

Удаление вакансий-дубликатов. Сохранение объединенных данных.

In [103]:
import pandas as pd

In [104]:
df1 = pd.read_csv('data/hh_data_part1.csv', sep='\t')
df2 = pd.read_csv('data/hh_data_part2.csv', sep='\t')

In [105]:
print(df1.shape)
print(df2.shape)

(1222, 22)
(2000, 22)


In [106]:
df = pd.concat([df1, df2])
df.drop_duplicates(['id'], inplace=True)
print(df.shape)

(2960, 22)


In [107]:
df.to_csv('data/hh_data.csv', sep='\t', index=False)

In [108]:
!ls -lh data

total 98M
-rw-rw-r-- 1 mariya mariya  11M сен 30 20:55 hh_data.csv
-rw-rw-r-- 1 mariya mariya 4,4M сен 30 20:52 hh_data_part1.csv
-rw-rw-r-- 1 mariya mariya 6,8M сен 30 20:52 hh_data_part2.csv
-rw-rw-r-- 1 mariya mariya  32M сен 30 20:45 hh_scrapped_data_part1.txt
-rw-rw-r-- 1 mariya mariya  45M сен 30 20:50 hh_scrapped_data_part2.txt
